### Command for bulding the localy cloned package

In [ ]:
# !pip install -e .

### Example

In [1]:
import torch
from normalizing_flows.flows import Flow, PrincipalManifoldFlow
from normalizing_flows.bijections import RealNVP

In [2]:
# Create some synthetic training and validation data
torch.manual_seed(0)

event_shape = (10,)
n_train = 100
n_val = 20
n_test = 10000

x_train = torch.randn(n_train, *event_shape) * 2 + 4
x_val = torch.randn(n_val, *event_shape) * 2 + 4
x_test = torch.randn(n_test, *event_shape) * 2 + 4

In [3]:
# Train without validation data
torch.manual_seed(0)
flow0 = PrincipalManifoldFlow(RealNVP(event_shape, n_layers=5), 10)
flow0.fit(x_train, show_progress=True)

Fitting Principal Manifold Flow: 100%|██████████| 500/500 [11:49<00:00,  1.42s/it, Training loss (batch): -inf]


In [4]:
print(f"Without validation data: {torch.mean(-flow0.log_prob(x_test))}")

Without validation data: 75.92913818359375


### Example case for understanding dimensions

In [3]:
def exp_reducer(x):
    return x.exp() + 1
inputs = torch.rand(10, 3)
print('inputs shape', inputs.shape)
M = torch.autograd.functional.jacobian(exp_reducer, inputs)
print('M shape', M.shape)

NameError: name 'torch' is not defined

In [11]:
A = exp_reducer(inputs)
print('A shape', A.shape)
print("input shape", inputs.shape)

A shape torch.Size([10, 3])
input shape torch.Size([10, 3])
